<a href="https://colab.research.google.com/github/etappau/info5731/blob/master/Paul_selfstudy12_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv

--2019-04-17 21:03:01--  https://raw.githubusercontent.com/wshuyi/demo-customer-churn-ann/master/customer_churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 684858 (669K) [text/plain]
Saving to: ‘customer_churn.csv.1’

customer_churn.csv. 100%[===================>] 668.81K  --.-KB/s    in 0.05s   

2019-04-17 21:03:01 (12.6 MB/s) - ‘customer_churn.csv.1’ saved [684858/684858]



In [0]:
import pandas as pd

In [0]:
df = pd.read_csv("customer_churn.csv")

In [0]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [0]:
df.tail()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1
9999,10000,15628319,Walker,792,France,Female,28,4,130142.79,1,1,0,38190.78,0


In [0]:
from sklearn.preprocessing import MinMaxScaler

In [0]:
df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']] = MinMaxScaler().fit_transform(df[['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']])

In [0]:
df.CreditScore

In [0]:
df.Age

In [0]:
!pip install -q tensorflow==2.0.0-alpha0

https://www.tensorflow.org/alpha

In [0]:
import tensorflow as tf
from tensorflow import keras

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train, test = train_test_split(df, test_size=0.2, random_state=1)

In [0]:
len(train)

8000

In [0]:
len(test)

2000

In [0]:
train, valid = train_test_split(train, test_size=0.2, random_state=1)

In [0]:
len(train)

6400

In [0]:
len(valid)

1600

Note that the quantities used for normalizing the test data are computed using the training data. You should never use in your workflow any quantity computed on thetest data, even for something as simple as data normalization.

In [0]:
from tensorflow import feature_column

In [0]:
tf.random.set_seed(1)

In [0]:
df.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [0]:
numeric_columns = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
categorical_columns = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']

In [0]:
feature_columns = []

In [0]:
for header in numeric_columns:
  feature_columns.append(feature_column.numeric_column(header))

In [0]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [0]:
def get_one_hot_rom_categorical(colname):
  categorical = feature_column.categorical_column_with_vocabulary_list(
  colname,
  train[colname].unique().tolist())
  return feature_column.indicator_column(categorical)

In [0]:
train["Geography"].unique().tolist()

['France', 'Spain', 'Germany']

In [0]:
get_one_hot_rom_categorical("Geography")

IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0))

In [0]:
for col in categorical_columns:
  feature_columns.append(get_one_hot_rom_categorical(col))

In [0]:
feature_columns

[NumericColumn(key='CreditScore', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Tenure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='Balance', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='NumOfProducts', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='EstimatedSalary', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Geography', vocabulary_list=('France', 'Spain', 'Germany'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='Gender', vocabulary_list=('Male', 'Female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(c

In [0]:
from tensorflow.keras import layers

In [0]:
feature_layer = layers.DenseFeatures(feature_columns)

In [0]:
feature_layer

In [0]:
model = keras.Sequential()
model.add(feature_layer)
model.add(layers.Dense(200, activation='relu'))
model.add(layers.Dense(100, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

In [0]:
def df_to_tfdata(df, shuffle=True, bs=32):
  df = df.copy()
  labels = df.pop('Exited')
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(df), seed=1)
  ds = ds.batch(bs)
  return(ds)

In [0]:
train_ds = df_to_tfdata(train)

In [0]:
valid_ds = df_to_tfdata(valid, shuffle=False)

In [0]:
test_ds = df_to_tfdata(test, shuffle=False)

In [0]:
model.fit(train_ds,
         validation_data = valid_ds,
         epochs=5)

Epoch 1/5
200/200 [==============================] - 6s 29ms/step - loss: 0.4689 - accuracy: 0.7947 - val_loss: 0.4470 - val_accuracy: 0.8044
Epoch 2/5
200/200 [==============================] - 6s 28ms/step - loss: 0.4311 - accuracy: 0.8141 - val_loss: 0.4276 - val_accuracy: 0.8206
Epoch 3/5
200/200 [==============================] - 6s 28ms/step - loss: 0.4090 - accuracy: 0.8304 - val_loss: 0.4099 - val_accuracy: 0.8231
Epoch 4/5
200/200 [==============================] - 6s 29ms/step - loss: 0.3817 - accuracy: 0.8384 - val_loss: 0.3853 - val_accuracy: 0.8375
Epoch 5/5
200/200 [==============================] - 6s 28ms/step - loss: 0.3570 - accuracy: 0.8501 - val_loss: 0.3760 - val_accuracy: 0.8425


In [0]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_1 (DenseFeatu multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  3200      
_________________________________________________________________
dense_4 (Dense)              multiple                  20100     
_________________________________________________________________
dense_5 (Dense)              multiple                  101       
Total params: 23,401
Trainable params: 23,401
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(test_ds)

W0417 21:33:19.232807 139802541643648 training_utils.py:1353] Expected a shuffled dataset but input dataset `x` is not shuffled. Please invoke `shuffle()` on input dataset.


63/63 [==============================] - 1s 23ms/step - loss: 0.3475 - accuracy: 0.8535


In [0]:
train.Exited.value_counts()

0    5108
1    1292
Name: Exited, dtype: int64

In [0]:
valid.Exited.value_counts()

0    1270
1     330
Name: Exited, dtype: int64

In [0]:
test.Exited.value_counts()

0    1585
1     415
Name: Exited, dtype: int64

In [0]:
model.evaluate(test_ds)

63/63 [==============================] - 1s 15ms/step - loss: 0.3346 - accuracy: 0.8645


[0.33464207512045663, 0.8645]

In [0]:
preds = model.predict(test_ds)

In [0]:
y_pred = (preds > 0.5).astype(np.float32)
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [0]:
from sklearn import metrics

In [0]:
y_test = test.Exited.values.astype(np.float32)

In [0]:
metrics.accuracy_score(y_test, y_pred)

0.8645

In [0]:
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      0.98      0.92      1585
         1.0       0.86      0.41      0.56       415

   micro avg       0.86      0.86      0.86      2000
   macro avg       0.86      0.70      0.74      2000
weighted avg       0.86      0.86      0.85      2000



In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
confusion_matrix(y_test, y_pred)

array([[1557,   28],
       [ 243,  172]])